In [113]:
import time
import sqlite3
import requests
import datetime
import pandas as pd
import shioaji as sj
import pickle


from exchange_calendars import get_calendar
import mplfinance as mpf
#import talib

from FuncBase import *

In [114]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('API_KEY'),os.getenv('SECRET_KEY'),  
          contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [115]:
api.activate_ca(
    ca_path = '/Users/ray881116/Documents/Ray_M/ekey/551/F229985348/S/Sinopac.pfx',
    ca_passwd = os.getenv('YOUR_CA_PASSWORD'),
    person_id = os.getenv('YOUR_PERSON_ID'),
)

True

<SecurityType.Index: 'IND'> fetch done


整合

In [116]:
from shioaji import TickSTKv1, Exchange
from shioaji import TickFOPv1, Exchange

#設定callback 回傳資料型態
@api.on_tick_stk_v1()
def stock_quote_callback(exchange: Exchange, tick:TickSTKv1):

    #full_code = msg['content'][0]  # 假設這是完整的商品代號
    #code = full_code[:3] 

    ts = pd.to_datetime(tick.datetime, format='%Y-%m-%d %H:%M:%S')

    #f tick.code[0].isalpha():
    #    code = tick.code[0:3]
    #else:
    code = tick.code

    ts = pd.to_datetime(tick.datetime)


    if 5 < ts.hour < 9:
        return
    
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    #print(close)
    #print(volume)
    #print(vol_sum)

    # 確保代號在 ticks 中存在
    #if code not in ticks:
    #    ticks[code] = pd.DataFrame(columns=["close", "volume"])

    # 更新 ticks 和 volume_today 資料
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum
    print(ticks[code].tail(10))
    #print(volume_today)

@api.on_tick_fop_v1()
def fut_quote_callback(exchange: Exchange, tick: TickFOPv1):

    ts = pd.to_datetime(tick.datetime)

    if tick.code[0].isalpha():
        tick.datetime
        code = tick.code[0:3]
    else:
        #print('tick.code is not only str')
        code = tick.code
        

    ts = pd.to_datetime(tick.datetime)

    if 5 < ts.hour < 9:
        return
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum


def place_cb(stat, msg):

    if stat.value == 'TFTDEAL':

        code = msg['code']
        qty = int(msg['quantity'])
                
        order_qty[code] -= qty #扣除下單時委託張數

        if order_qty[code] == 0:
            open_orders = None #委託張數歸0

        position = positions.get(code,0)

        if msg['action'] == 'Sell':
            qty = qty * -1

        position += qty

        positions[code] = position
    
api.set_order_callback(place_cb)


In [117]:
'''
code = None
entry_time = None
entry_price = 0

exit_time = None
exit_price = 0

AC = 0
EH = 0
last_price = 0
cost_after_add = 0

with open('trade_record.pkl', 'wb') as file:
    tr = {}

    pickle.dump(tr, file)

print("資料已保存")
'''

'\ncode = None\nentry_time = None\nentry_price = 0\n\nexit_time = None\nexit_price = 0\n\nAC = 0\nEH = 0\nlast_price = 0\ncost_after_add = 0\n\nwith open(\'trade_record.pkl\', \'wb\') as file:\n    tr = {}\n\n    pickle.dump(tr, file)\n\nprint("資料已保存")\n'

In [118]:
'''
import time
import pickle

def tbot(codes, is_Futures = False,
         filter = 0.1,
         Ratio_Drawdown1 = 0.1,
         Ratio_Drawdown2 = -0.03,
         Quit_position = 0
         ):

    print('--- 啟動 Tbot for {} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')

    #Risk Variables
    filter = filter
    Ratio_Drawdown1 = Ratio_Drawdown1 #預設台股最大漲跌幅
    Ratio_Drawdown2 = Ratio_Drawdown2
    Quit_position = Quit_position

    #Call Other variable
    with open('trade_historical_variable.pkl', 'rb') as file:
        data = pickle.load(file)

    entry_time = data['Entry_time']
    entry_price = data['Entry_P']
    last_price = data['last_P']
    AC = data['AC']
    EH = data['EH']
    cost_after_add = data['cost_after_add']
    
    
    #Start
    dates = pd.to_datetime(datetime.datetime.now())

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = pd.to_datetime(dates.date()+datetime.timedelta(days=1))
        
        else:
            date = pd.to_datetime(dates.date())
    print(date)
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return

    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    
    else:
        code = codes

    ticks[code] = get_ticks(connection, api, date, codes ,is_Futures)[0]
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = get_MA(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][prev_trading_date:]
    #print(kbars[code][(datetime.datetime.now()-datetime.timedelta(minutes=1).strftime('%Y-%m-%d %H:%M:%S'))])

    volume_today[code] = kbars[code]['volume'].sum()

    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(code)[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[codes],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )
    
    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()


        if  current_time.second == 0:

            kbars[code] = get_MA((ticks_to_kbars(ticks[code], interval='1Min')))
            kbars[code] = kbars[code][prev_trading_date:]


            str_current_time = (current_time - datetime.timedelta(minutes=30)).strftime('%Y-%m-%d %H:%M:%S')
                
            try:
                    
                current_price = kbars[code].loc[str_current_time]['close']
                
                if current_price >= EH:
                    EH = current_price
            except:
               continue
            
        
            open_order = open_orders.get(code, None) 
        

            # Check the order status
            if open_order:

                api.update_status(api.stock_account)

                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:

                   api.cancel_order(open_order)

                   open_orders[code] = None
                   order_qty[code] = 0

                if open_order.status.status != 'Filling':
                    break

            position = positions.get(code, 0)

            if (
                #first condition
                position == 0 and 
                #OR condition
                #current_price/kbars.iloc[ts]['ma20']-1 < filter or 
                current_price > kbars[code].loc[str_current_time]['ma20']                        
            ):
                if (
                    #Entry condition
                    current_price > kbars[code].loc[str_current_time]['ma10']         
                ):
                    qty = 1 #改成金額決定股數

                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Buy",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = qty

                    entry_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    entry_price = current_price

                    #Cost record
                    AC = current_price
                    EH = current_price
                    last_price = current_price
                    cost_after_add = current_price
                    
                    #Report
                    print( 'Entry: [{}] buy {} at {}'.format(current_time, codes, current_price))

            elif(
                position != 0
            ):
                add_position = 1
                cost_after_add = (AC * position + current_price * add_position) / (position + add_position)
                cv1 = EH * (1+Ratio_Drawdown1) / cost_after_add -1
                cv2 = EH * (1 + Ratio_Drawdown1) 
                

                #兩個加碼條件都滿足才加碼
                # 加碼條件1
                if cv1 > 0:
                    if cv2 > last_price:
                        qty = add_position

                        contract = api.Contracts.Stocks[code]

                        order = api.Order(price=current_price,
                            quantity=qty,
                            action="Buy",
                            price_type="LMT",
                            order_type="ROD",
                            order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                            account=api.stock_account)

                        trade = api.place_order(contract, order)

                        open_orders[code] = trade
                        order_qty[code] = qty

                        #Cost record
                        AC = (AC * (position-add_position) + current_price * add_position) / position
                        EH = current_price
                        last_price = current_price

                            
                        #Report
                        print( 'Entry: [{}] Reach adding condition: {} and buy {} at {}'.format(current_time,  round(cv1, 5),
                                                                                        codes, current_price))
                        print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))

                    #出場條件1
                elif(
                    position < 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    
                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Sell",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)


                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position

                    print('Exit: [{}] Reach condition1: {} and sell {} for {} units at {}'.format(current_time,
                                                                                                   round(EH * (1+Ratio_Drawdown1), 5),
                                                                                                  codes, position, current_price))
                    
                elif(
                    position >= 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Sell",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)


                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = position

                    print( 'Exit: [{}] Reach condition2: {} and sell {} for {} units at {}'.format(current_time, 
                                                                                                  round(EH * (1+Ratio_Drawdown2),5),
                                                                                                  codes, position, current_price))
                break
    if is_Futures:
        api.quote.unsubscribe(api.Contracts.Futures.get(code)[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
            

    else:
        api.quote.unsubscribe(api.Contracts.Stocks[codes],
                        quote_type= 'tick',
                        version = sj.constant.QuoteVersion.v1
                            )
    


    with open('trade_historical_variable.pkl', 'wb') as file:
        trade_his_var = {
                        'Entry_time': entry_time,
                        'Entry_P':entry_price,
                        'last_P': last_price,
                        'AC': AC,
                        'EH': EH,
                        'cost_after_add':cost_after_add,
                        'Position': positions.get(code, 0)}

        pickle.dump(trade_his_var, file)

    print("資料已保存")
        
    print('----- 關閉 Tbot  #{} -----'.format(code))    
    '''                                                                             

'\nimport time\nimport pickle\n\ndef tbot(codes, is_Futures = False,\n         filter = 0.1,\n         Ratio_Drawdown1 = 0.1,\n         Ratio_Drawdown2 = -0.03,\n         Quit_position = 0\n         ):\n\n    print(\'--- 啟動 Tbot for {} ---\'.format(codes))\n\n    tw_calendar = get_calendar(\'XTAI\')\n\n    #Risk Variables\n    filter = filter\n    Ratio_Drawdown1 = Ratio_Drawdown1 #預設台股最大漲跌幅\n    Ratio_Drawdown2 = Ratio_Drawdown2\n    Quit_position = Quit_position\n\n    #Call Other variable\n    with open(\'trade_historical_variable.pkl\', \'rb\') as file:\n        data = pickle.load(file)\n\n    entry_time = data[\'Entry_time\']\n    entry_price = data[\'Entry_P\']\n    last_price = data[\'last_P\']\n    AC = data[\'AC\']\n    EH = data[\'EH\']\n    cost_after_add = data[\'cost_after_add\']\n    \n    \n    #Start\n    dates = pd.to_datetime(datetime.datetime.now())\n\n    if is_Futures:\n        \n        #若為期貨只取英文代號作為code name\n        code = \'\'.join(char for char in codes if cha

<SecurityType.Future: 'FUT'> fetch done


Multiple tbot(未下單)

In [119]:
ticks = {}
kbars = {}
volume_today = {}

open_orders = {}
order_qty = {}
positions = {}

workers = {}
stop_threads = False


In [120]:
import time
import pickle

def M_tbot(codes, is_Futures = False,
           enforcement = False,
         filter = 0.08,
         Ratio_Drawdown1 = -0.05,
         Ratio_Drawdown2 = -0.03,
         Quit_position = 0
         ):

    print('--- 啟動 Tbot for {} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')

    #Risk Variables
    filter = filter
    Ratio_Drawdown1 = Ratio_Drawdown1 #預設台股最大漲跌幅
    Ratio_Drawdown2 = Ratio_Drawdown2
    Quit_position = Quit_position

    '''
    #Call Other variable
    with open('trade_historical_var.pkl', 'rb') as file:
        trade_his_var = pickle.load(file)

    code = trade_his_var['code']
    entry_time = trade_his_var['Entry_time']
    entry_price = trade_his_var['Entry_P']
    last_price = trade_his_var['last_P']
    AC = trade_his_var['AC']
    EH = trade_his_var['EH']
    cost_after_add = trade_his_var['cost_after_add']
    '''
    
    #Start
    dates = pd.to_datetime(datetime.datetime.now())

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = pd.to_datetime(dates.date()+datetime.timedelta(days=1))
        
        else:
            date = pd.to_datetime(dates.date())
    
    else:
        code = codes
        date = pd.to_datetime(dates.date())

    prev_trading_date = tw_calendar.previous_close(date).date()

    #若calendar日期有誤可強制執行交易
    if enforcement:
        print("啟用強制執行")
    else:
        if date not in tw_calendar.opens:
            print("今日非交易日")
            return
        
    # Retrieve and print the last trade record for code
    try:
        with open('trade_record.pkl', 'rb') as file:
            trade_his_var = pickle.load(file)
    except (FileNotFoundError, EOFError):
        trade_his_var = {}
    
    print(trade_his_var)
    
    last_trade = get_last_trade_by_code(trade_his_var, code)

    if last_trade:
        entry_time = last_trade['Entry_time']
        entry_price = last_trade['Entry_price']
        last_price = last_trade['last_price']
        AC = last_trade['AC']
        EH = last_trade['EH']
        cost_after_add = last_trade['cost_after_add']
        position = last_trade['Position']

        print(f"最後一筆交易紀錄：標的 {code}, 進場時間 {last_trade['Entry_time']}, 進場價 {last_trade['Entry_price']}, 最後價 {last_trade['last_price']}")
    else:
        entry_time = None
        entry_price = 0

        exit_time = None
        exit_price = 0
        last_price = 0
        AC = 0
        EH = 0
        cost_after_add = 0
        position =0 
        
        print(f"未找到代碼 {code} 的交易紀錄")


    #Get historical ticks and turn into kbars
    ticks[code] = pd.concat([get_ticks(connection, api, date, codes , is_Futures )[0],
                              get_ticks(connection, api, prev_trading_date, codes , is_Futures )[0]])

    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = get_MA(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][prev_trading_date:]
    volume_today[code] = kbars[code]['volume'].sum()

    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(code)[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[code],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )
    
    while True:
        if stop_threads:
            break

        time.sleep(1)

        current_time = datetime.datetime.now()
        if  current_time.second == 0:
            
            kbars[code] = get_MA((ticks_to_kbars(ticks[code], interval='1Min')))
            kbars[code] = kbars[code][prev_trading_date:]
            #print(kbars[code].tail(5))



            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')
   
            try:        
                current_price = kbars[code].loc[str_current_time]['close']
                
                if current_price >= EH:
                    EH = current_price
            except:
               continue
            
        
            open_order = open_orders.get(code, None) 
        

            # Check the order status
            if open_order:

                api.update_status(api.stock_account)

                if open_order.status.status in ['PendingSubmit', 'Submitted', 'Filling']:

                   api.cancel_order(open_order)

                   open_orders[code] = None
                   order_qty[code] = 0

                if open_order.status.status != 'Filling':
                    break
            
            print(f"{current_time}: {code}'s position = {position}")

            if (
                #first condition
                position == 0 and 
                #OR condition
                #current_price/kbars.iloc[ts]['ma20']-1 < filter or 
                current_price > kbars[code].loc[str_current_time]['ma20']                        
            ):
                if (
                    #Entry condition
                    current_price > kbars[code].loc[str_current_time]['ma10']         
                ):
                    qty = 1 #改成金額決定股數
                    position = qty

                    '''
                    contract = api.Contracts.Stocks[code]

                    order = api.Order(price=current_price,
                        quantity=qty,
                        action="Buy",
                        price_type="LMT",
                        order_type="ROD",
                        order_lot=sj.constant.StockOrderLot.IntradayOdd, 
                        account=api.stock_account)

                    trade = api.place_order(contract, order)

                    open_orders[code] = trade
                    order_qty[code] = qty
                    '''

                    entry_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    entry_price = current_price

                    #Cost record
                    AC = current_price
                    EH = current_price
                    last_price = current_price
                    cost_after_add = current_price
                    
                    #Report
                    print( 'Entry: [{}] buy {} at {}.'.format(current_time, codes, current_price))

            elif(
                position != 0
            ):
                #記錄進場後(不論有沒有進行加減碼)的 New Entry High 
                if current_price > EH:
                    EH = current_price

                add_position = 1
                cv1 = EH * (1+Ratio_Drawdown1) / cost_after_add -1
                cv2 = EH * (1 + Ratio_Drawdown1) 
                

                #兩個加碼條件都滿足才加碼
                # 加碼條件1
                if cv1 > 0:
                    if cv2 > last_price:
                        qty = add_position

                        position = position+qty 

                        #Cost record
                        AC = (AC * (position-add_position) + current_price * add_position) / position
                        EH = current_price
                        last_price = current_price
                        cost_after_add = (AC * (position-add_position) + current_price * add_position) / position


                            
                        #Report
                        print( 'Entry: [{}] Reach adding condition: {} and buy {} at {}'.format(current_time,  round(cv1, 5),
                                                                                        codes, current_price))
                        print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))

                    #出場條件1
                elif(
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    if position < 5: 
                        position= 0
                        cost_after_add=0

                        print('Exit: [{}] Reach condition1: {} and sell {} for {} units at {}'.format(current_time,
                                                                                                    round(EH * (1+Ratio_Drawdown1), 5),
                                                                                                    codes, position, current_price))
                        
                    elif(
                        position >= 5
                    ):
                        position = 0
                        cost_after_add=0
                                                
                        print( 'Exit: [{}] Reach condition2: {} and sell {} for {} units at {}'.format(current_time, 
                                                                                                    round(EH * (1+Ratio_Drawdown2),5),
                                                                                                    codes, position, current_price))
                    break
            
            else:
                # If neither the add position nor exit conditions are met, continue
                pass
            continue
        elif (current_time.hour >=14):
            break
        
    
    if is_Futures:
        api.quote.unsubscribe(api.Contracts.Futures.get(code)[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
            

    else:
    
        api.quote.unsubscribe(api.Contracts.Stocks[code],
                        quote_type= 'tick',
                        version = sj.constant.QuoteVersion.v1
                            )
        
    #save transaction record
    update_trade_record(code, entry_time, entry_price, last_price, AC, EH, cost_after_add, position)
    
    

    '''
    with open('trade_historical_var.pkl', 'wb') as file:
        trade_his_var = {
                        'code': code,
                        'Entry_time': entry_time,
                        'Entry_P':entry_price,
                        'last_P': last_price,
                        'AC': AC,
                        'EH': EH,
                        'cost_after_add':cost_after_add,
                        'Position': positions.get(code, 0)
                        }

        pickle.dump(trade_his_var, file)

    print("資料已保存")
    '''
        
    print('----- 關閉 Tbot  #{} -----'.format(code))                                                                                 

In [121]:
import threading
import datetime
import pandas as pd

def run(daily_target, is_Futures = False, enforcement=False):

    #date = pd.to_datetime(datetime.datetime.now().date())
    #workers = {}

    print('今日當沖標的：{}'.format(daily_target))

    for code in daily_target:

        worker = threading.Thread(name=code, target= M_tbot, args=(code,is_Futures,enforcement))
        worker.start()
        workers[code] = worker
    

In [123]:
daily_target = ['6214','2454']
update_historical_data('2024-12-23','2024-12-24', daily_target, connection, api)

{Timestamp('2024-12-23 00:00:00'): ['6214', '2454'], Timestamp('2024-12-24 00:00:00'): ['6214', '2454']}
正在更新每日收盤行情...
2024-12-20 00:00:00
20241220:Data exist.
2024-12-23 00:00:00
20241223:Data exist.
2024-12-24 00:00:00
20241224 找不到資料
正在更新逐筆交易...
正在更新2024/12/23逐筆成交資料，總共2檔標的，為['6214', '2454']
正在更新2024/12/24逐筆成交資料，總共2檔標的，為['6214', '2454']
Data stored successfully.
股市資料更新完成


In [124]:
run(daily_target,enforcement=True)

今日當沖標的：['6214', '2454']
--- 啟動 Tbot for 6214 ---
啟用強制執行
--- 啟動 Tbot for 2454 ---
啟用強制執行
{'2317': {'Entry_time': Timestamp('2024-12-23 09:02:00'), 'Entry_price': 184.5, 'last_price': 184.5, 'AC': 184.5, 'EH': 187.0, 'cost_after_add': 185.0, 'Position': 1}, '2330': {'Entry_time': Timestamp('2024-12-23 09:02:00'), 'Entry_price': 1065.0, 'last_price': 1065.0, 'AC': 1065.0, 'EH': 1080.0, 'cost_after_add': 1072.5, 'Position': 1}}
未找到代碼 6214 的交易紀錄
{'2317': {'Entry_time': Timestamp('2024-12-23 09:02:00'), 'Entry_price': 184.5, 'last_price': 184.5, 'AC': 184.5, 'EH': 187.0, 'cost_after_add': 185.0, 'Position': 1}, '2330': {'Entry_time': Timestamp('2024-12-23 09:02:00'), 'Entry_price': 1065.0, 'last_price': 1065.0, 'AC': 1065.0, 'EH': 1080.0, 'cost_after_add': 1072.5, 'Position': 1}}
未找到代碼 2454 的交易紀錄


2024-12-24 10:09:00.409994: 2454's position = 0
2024-12-24 10:09:00.475500: 6214's position = 0
2024-12-24 10:10:00.657388: 2454's position = 0
Entry: [2024-12-24 10:10:00.657388] buy 2454 at 1425.0.
2024-12-24 10:10:00.798763: 6214's position = 0
2024-12-24 10:11:00.044573: 6214's position = 0
2024-12-24 10:12:00.146970: 2454's position = 1
2024-12-24 10:12:00.274856: 6214's position = 0
2024-12-24 10:13:00.384974: 2454's position = 1
2024-12-24 10:13:00.510147: 6214's position = 0
Entry: [2024-12-24 10:13:00.510147] buy 6214 at 142.5.
2024-12-24 10:14:00.697395: 2454's position = 1
2024-12-24 10:14:00.814045: 6214's position = 1
2024-12-24 10:15:00.148376: 2454's position = 1
2024-12-24 10:15:00.261802: 6214's position = 1
2024-12-24 10:16:00.726565: 6214's position = 1
2024-12-24 10:17:00.054157: 2454's position = 1
2024-12-24 10:17:00.166353: 6214's position = 1
2024-12-24 10:18:00.474243: 2454's position = 1
2024-12-24 10:18:00.628688: 6214's position = 1
2024-12-24 10:19:00.05124

In [125]:
workers

{'6214': <Thread(6214, started 13130186752)>,
 '2454': <Thread(2454, started 13146976256)>}

In [126]:
ticks
kbars
#volume_today

{'2454':                        open   close    high     low  volume     ma5    ma10  \
 ts                                                                            
 2024-12-23 09:00:00  1405.0  1410.0  1415.0  1405.0     219     NaN     NaN   
 2024-12-23 09:01:00  1410.0  1415.0  1415.0  1410.0      15     NaN     NaN   
 2024-12-23 09:02:00  1415.0  1410.0  1415.0  1410.0      24     NaN     NaN   
 2024-12-23 09:03:00  1410.0  1410.0  1410.0  1410.0      26     NaN     NaN   
 2024-12-23 09:04:00  1405.0  1405.0  1410.0  1400.0      80  1410.0     NaN   
 ...                     ...     ...     ...     ...     ...     ...     ...   
 2024-12-24 10:04:00  1420.0  1420.0  1420.0  1420.0       2  1423.0  1423.0   
 2024-12-24 10:05:00  1420.0  1420.0  1420.0  1420.0       1  1422.0  1423.0   
 2024-12-24 10:06:00  1425.0  1425.0  1425.0  1425.0       1  1423.0  1423.0   
 2024-12-24 10:07:00  1425.0  1425.0  1425.0  1420.0       5  1423.0  1423.0   
 2024-12-24 10:08:00  1420.0  14

In [92]:
stop_threads = True

In [24]:
api.quote.unsubscribe(api.Contracts.Stocks['2317'],
                        quote_type= 'tick',
                        version = sj.constant.QuoteVersion.v1
                            )

In [35]:
with open('trade_record.pkl', 'rb') as file:
    trade_his_var = pickle.load(file)

# 確認載入的資料
print(trade_his_var)

{'2317': {'Entry_time': Timestamp('2024-12-19 11:38:00'), 'Entry_price': 180.5, 'last_price': 180.5, 'AC': 180.5, 'EH': 181.5, 'cost_after_add': 180.75, 'Position': 1}, '2330': {'Entry_time': Timestamp('2024-12-19 11:42:00'), 'Entry_price': 1060.0, 'last_price': 1060.0, 'AC': 1060.0, 'EH': 1070.0, 'cost_after_add': 1065.0, 'Position': 1}}


In [98]:
code='2317'

In [99]:
try:
        with open('trade_record.pkl', 'rb') as file:
            trade_his_var = pickle.load(file)
except (FileNotFoundError, EOFError):
        trade_his_var = {}
    
last_trade = get_last_trade_by_code(trade_his_var, code)

In [100]:
last_trade

{'Entry_time': Timestamp('2024-12-23 09:02:00'),
 'Entry_price': 184.5,
 'last_price': 184.5,
 'AC': 184.5,
 'EH': 187.0,
 'cost_after_add': 185.0,
 'Position': 1}

In [102]:
if last_trade:
        entry_time = last_trade['Entry_time']
        entry_price = last_trade['Entry_price']
        last_price = last_trade['last_price']
        AC = last_trade['AC']
        EH = last_trade['EH']
        cost_after_add = last_trade['cost_after_add']
        position = last_trade['Position']

        print(f"最後一筆交易紀錄：代碼 {code}, 進場時間 {last_trade['Entry_time']}, 進場價 {last_trade['Entry_price']}, 最後價 {last_trade['last_price']}")

最後一筆交易紀錄：代碼 2317, 進場時間 2024-12-23 09:02:00, 進場價 184.5, 最後價 184.5


In [ ]:
data = {
    '2317': {
        'Entry_time': '2024-12-19 11:38:00',
        'Entry_price': 180.5,
        'last_price': 180.5,
        'AC': 180.5,
        'EH': 181.5,
        'cost_after_add': 180.75,
        'Position': 1
    },
    '2330': {
        'Entry_time': '2024-12-19 11:42:00',
        'Entry_price': 1060.0,
        'last_price': 1060.0,
        'AC': 1060.0,
        'EH': 1070.0,
        'cost_after_add': 1065.0,
        'Position': 1
    }
}
file_path = "trade_record.pkl"  # 檔案名稱可以任意修改
with open(file_path, "wb") as file:  # "wb" 表示以二進制模式寫入
    pickle.dump(data, file)          # 使用 pickle.dump() 將資料寫入檔案
